Start jupyter-lab

```bash
jupyter-lab --notebook-dir=$HOME/gits/gerashegalov/rapids-shell/src/jupyter
```
or simply open in VS Code with Jupyter extension

# Repro for [NVIDIA/spark-rapids#8625](https://github.com/NVIDIA/spark-rapids/issues/8625)

In [ ]:
import datetime
import findspark
import glob
import json
import os
import pyarrow
import pyarrow.parquet

### Versions

In [ ]:
rapids_version = '23.08.0-SNAPSHOT'
spark_version = '3.4.1'
cuda_version = 'cuda11'
scala_version = '2.12'

### Environment Variables

In [ ]:
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_LOCAL_IP'] = '127.0.0.1'
os.environ['TZ'] = 'UTC'

In [ ]:
home_dir = os.environ['HOME']
work_dir = f"{home_dir}/jupyter_run_dir"
rapids_home = f"{home_dir}/gits/NVIDIA/spark-rapids"
dist_jar = f"rapids-4-spark_{scala_version}-{rapids_version}-{cuda_version}.jar"
spark_home = f"{home_dir}/dist/spark-{spark_version}-bin-hadoop3"

### Find Spark 

In [ ]:
findspark.init(spark_home = spark_home)
findspark.add_jars(f"{rapids_home}/dist/target/{dist_jar}")
import pyspark

### Configure Spark

In [ ]:
cores_per_exec = 1
spark_master = f"local[{cores_per_exec}]"
spark_builder = pyspark.sql.SparkSession.builder.config(
    map = {
        'spark.app.name': 'RAPIDS PySpark Notebook',
        'spark.driver.extraJavaOptions': '-Dai.rapids.cudf.preserve-dependencies=true',
        'spark.master': spark_master,
        'spark.plugins': 'com.nvidia.spark.SQLPlugin',
        'spark.rapids.sql.enabled': False,
        'spark.rapids.sql.explain': 'ALL', 
    }   
)
spark = spark_builder.getOrCreate()

# Test Scenario

In [ ]:
ts_tup = (
    datetime.datetime(2023, 7, 14, 7, 38, 45, 418688),
    datetime.datetime(5023, 7, 14, 7, 38, 45, 418688),
)
ts_tup

In [ ]:
spark.createDataFrame([ts_tup,], 'ts timestamp, big_ts timestamp').createOrReplaceTempView('df1')
spark.sql("SELECT * FROM df1").show(truncate = False)
spark.sql("SELECT * FROM df1").printSchema()

In [ ]:
spark.conf.get('spark.sql.parquet.outputTimestampType')

In [ ]:
spark.sql("SELECT MAX(ts) as max_ts, MAX(big_ts) as max_big_ts FROM df1").write.mode('overwrite').parquet('/tmp/int96_out.parquet')

In [ ]:
read_df = spark.read.parquet('/tmp/int96_out.parquet')
read_df.printSchema()
read_df.show(truncate = False)

In [ ]:
parquet_file, = glob.glob('/tmp/int96_out.parquet/*.parquet')

In [ ]:
pmd = pyarrow.parquet.read_metadata(parquet_file)
pmd.to_dict()

In [ ]:
for k, v in pmd.metadata.items():
    print(k.decode('ascii'))
    print(v.decode('ascii'))

In [ ]:
pmd.schema

In [ ]:
pmd.row_group(0).column(0)        

In [ ]:
data = pyarrow.parquet.read_table(parquet_file)
data